# Simplified `Compressor`
We create a very simple compressor model and focus on functionalities : 
- simulation : fixed design and functional characteristics, performance evaluation
- calibration : geometry is known, functional characteristics are evaluated from given performance
- design : functional characteristics are known, geometry is evaluated from performance target

In [ ]:
from numpy import pi
from pyturbo.systems import CompressorSimpleAero
from cosapp.drivers import NonLinearSolver, RunSingleCase

cmp = CompressorSimpleAero('cmp')

## simulation
A compressor is like a mixer of two energy flux : the shaft and the fluid in are mergered in a fluid out. As for a mixer, we need to release a parameter to respect the behaviour of the compressor. 

We could liberate `fl_in.W` or `shaft_in.power`.

In [ ]:
run = cmp.add_driver(NonLinearSolver('run'))
run.add_unknown('shaft_in.power') # or run.add_unknown('shaft_in.power')

In [ ]:
cmp.fl_in.W = 400.
cmp.shaft_in.N = 5000.

cmp.run_drivers()

print('mass flow : ', cmp.fl_in.W,' kg/s')
print('shaft power : ', cmp.shaft_in.power / 1e6, ' MW')
print('pt_ratio : ', cmp.pt_ratio)
print('Tt_ratio : ', cmp.Tt_ratio)
print('phi : ', cmp.phi)
print('psi : ', cmp.psi)

### calibration
From a given geometry, we want to evaluate the functional caracteristics that provid the expected performances.

Reference is `CFM56-7B22` with geometrical values:
- fan diameter `61 inch`
- hub 2 tip radius `0.3`

Functional characteristics are: 
- phiP

Design characteristics are:
- pt_ratio_design
- Utip_design

Performances reference date used are:
- W = 330 kg/s
- shaft_N = 5100 rpm
- shaft_power = 10 MW


In [ ]:
# geometrical data
cmp.r_out_tip = cmp.r_in_tip = 61 * 0.0254 / 2
cmp.area_in = pi * cmp.r_in_tip ** 2 * (1 - 0.3 ** 2)

In [ ]:
# calibration case
run = cmp.add_driver(NonLinearSolver('run'))

calib = run.add_child(RunSingleCase('calib'))
calib.design.extend(cmp.design_methods['calib'])

In [ ]:
cmp.fl_in.W = 330.
cmp.shaft_in.N = 5100
cmp.shaft_in.power = 10e6

cmp.run_drivers()

print('shaft power : ', cmp.shaft_in.power / 1e6, ' MW')
print('pt_ratio : ', cmp.pt_ratio)
print('Tt_ratio : ', cmp.Tt_ratio)
print('phi : ', cmp.phi)
print('psi : ', cmp.psi)
print('Utip : ', cmp.Utip, ' m/s')

### design
geometry is evaluated to fit performance requirements from the same functional (adimensionalised) characteristics. Geometry is defined by : 
- te_tip_r
- area_in

In [ ]:
run = cmp.add_driver(NonLinearSolver('run'))

case = run.add_child(RunSingleCase('case'))
case.design.extend(cmp.design_methods['sizing'])

# addition conditions for lack of geometry
case.add_unknown('r_out_tip').add_unknown('area_in')
case.add_equation('area_in == pi * r_out_tip ** 2 * (1 - 0.3 ** 2)')

# addition conditions for isolated component
case.add_unknown('shaft_in.N').add_unknown('shaft_in.power')

In [ ]:
cmp.fl_in.W = 330.
cmp.run_drivers()

print('mass flow : ', cmp.fl_in.W,' kg/s')
print('shaft power : ', cmp.shaft_in.power / 1e6, ' MW')
print('shaft N : ', cmp.shaft_in.N, ' rpm')
print('pt_ratio : ', cmp.pt_ratio)
print('Tt_ratio : ', cmp.Tt_ratio)
print('r_out_tip : ', cmp.r_out_tip, ' m')

### exploitation

Component is designed from a calibrated technology to fit new functional requirements

In [ ]:
run = cmp.add_driver(NonLinearSolver('run'))
run.add_unknown('shaft_in.power') # or run.add_unknown('fl_in.W')

In [ ]:
cmp.fl_in.W = 330.
cmp.shaft_in.N = 5100.

cmp.run_drivers()

print('shaft power : ', cmp.shaft_in.power / 1e6, ' MW')
print('pt_ratio : ', cmp.pt_ratio)